In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd 'gdrive/My Drive/Colab Notebooks/CompVision-Course/Kaggle'

/content/gdrive/My Drive/Colab Notebooks/CompVision-Course/Kaggle


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


# Any results you write to the current directory are saved as output.

In [0]:

import torch
import random

random.seed(8)
np.random.seed(8)
torch.manual_seed(8)
torch.cuda.manual_seed(8)
torch.backends.cudnn.deterministic = True

In [0]:
data_root = 'plates/plates/'
print(os.listdir(data_root))

['.DS_Store', 'test', 'train']


In [0]:
import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

# Создаем папки для тренировки и валидации
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, filename in enumerate(tqdm(os.listdir(source_dir))):
        dest_dir = os.path.join(train_dir, class_name)
        shutil.copy(os.path.join(source_dir, filename), os.path.join(dest_dir, filename))


100%|██████████| 21/21 [00:17<00:00,  1.32it/s]


In [0]:
import random
from PIL import ImageOps
import torchvision.transforms.functional as F

def my_inverse_transformation(img, p=0.5):
    if not F._is_pil_image(img):
        raise TypeError('img should be PIL Image. Got {}'.format(type(img)))
    if random.random() > p:
        return ImageOps.invert(img)
    else:
        return img

In [0]:
import cv2
from PIL import Image
from torchvision.transforms.functional import _is_pil_image
from torchvision import transforms, models


class Segmentate:
    center_crop = transforms.CenterCrop(224)
    initThresh = 105

    def __init__(self):
        pass

    def __call__(self, img):
        if not _is_pil_image(img):
            raise TypeError('Img should be PIL Image. Got {}'.format(type(img)))

        cimg = img.copy()
        img = np.array(img)

        # Convert to gray-scale
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        p1 = self.initThresh
        p2 = self.initThresh * 0.4

        # Detect circles using HoughCircles transform
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 500, param1=p1, param2=p2, minRadius=10,
                                   maxRadius=170)

        t = 400
        if circles is None:
            return self.center_crop(cimg)

        c = np.uint16(np.around(circles))[0, 0]

        # Draw the outer circle
        cv2.circle(img, (c[0], c[1]), c[2] + t // 2 - 15, (0, 0, 0), t)

        thr = -10
        # Centering ad cropping
        try:
            img = img[c[1] - c[2] - thr:c[1] + c[2] + thr, c[0] - c[2] - thr:c[0] + c[2] + thr]
            pil_img = Image.fromarray(img)
        except ValueError:
            return cimg

        return pil_img



In [0]:
class ChangeContrast:
    def __init__(self, level):
        self.level = level
        
    def __call__(self, img):
        self.factor_ = (259 * (self.level + 255)) / (255 * (259 - self.level))
        def contrast(c):
            return 128 + self.factor_ * (c - 128)
        return img.point(contrast)
    
    def __repr__(self):
        return self.__class__.__name__ + '(contrast_level={})'.format(self.level)

In [0]:
import torch
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import PIL

shift_const = [0.485, 0.456, 0.406]
scale_const = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Pad(50, padding_mode='edge'),
    transforms.RandomRotation((0, 360), expand=True),
    transforms.CenterCrop(224),    
#     ChangeContrast(70),
#     Segmentate(),
#     transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
#     transforms.ColorJitter(0.5, 0.6, 0.4, 0.25),
#     transforms.Lambda(lambda x: my_inverse_transformation(x, 0.3)),
    transforms.ToTensor(),
#     transforms.Lambda(lambda x: x[np.random.permutation(3), :, :]),
    transforms.Normalize(shift_const, scale_const)
])

val_transforms = transforms.Compose([
    transforms.CenterCrop(224),
#     ChangeContrast(70),
#     Segmentate(),
#     transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(shift_const, scale_const)
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 20

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)


In [0]:
print(len(train_loader), len(train_dataset))
print(len(val_loader), len(val_dataset))

2 40
38 744


In [0]:
!pip install pretrainedmodels

In [0]:
def make_pnasnet():
    import pretrainedmodels
    model = pretrainedmodels.pnasnet5large(num_classes=1000)
    model.last_linear = torch.nn.Linear(model.last_linear.in_features, 2)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model


def make_xception():
    import pretrainedmodels
    model = pretrainedmodels.xception()
    model.last_linear = torch.nn.Linear(model.last_linear.in_features, 2)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model


def make_resnext():
    import pretrainedmodels
    model = pretrainedmodels.se_resnext50_32x4d()
    model.last_linear = torch.nn.Linear(model.last_linear.in_features, 2)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model



In [0]:
def make_densenet(freeze=False):
    import pretrainedmodels
    model = pretrainedmodels.densenet121()
#     filename = 'models/inceptionresnetv2.pth'
#     model = torch.load_state_dict(filename)
    if freeze:
        for param in model.parameters():
            param.requires_grad = False
    
    model.last_linear = torch.nn.Linear(model.last_linear.in_features, 2)
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model

In [0]:
def make_inception_resnet_v2(freeze=False):
    import pretrainedmodels
    model = pretrainedmodels.inceptionresnetv2()
#     filename = 'models/inceptionresnetv2.pth'
#     model = torch.load_state_dict(filename)
    if freeze:
        for param in model.parameters():
            param.requires_grad = False
    
    model.last_linear = torch.nn.Linear(model.last_linear.in_features, 2)
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model

In [0]:
def make_vgg():
    model = models.vgg19_bn(pretrained=True)
    child_counter = 0
    for child in model.features.children():
        if child_counter < 48:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
        
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model

In [0]:
def make_resnet():
    model = models.resnet18(pretrained=True)
#     for param in model.parameters():
#         param.requires_grad = False

#     child_counter = 0
#     for child in model.children():
#         if child_counter == 7:
#             child_of_child_counter = 0
#             for child_of_child in child.children():
#                 if child_of_child_counter == 1:
#                     for param in child_of_child.parameters():
#                         param.requires_grad = True
#                 child_of_child_counter += 1
#         child_counter += 1
        
    model.fc = torch.nn.Linear(model.fc.in_features,2)


    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model

In [0]:
from torchvision import models
from itertools import product

class GridSearch():
    def __init__(self, loss, optimizer=torch.optim.SGD, optimizer_params={}, scheduler=torch.optim.lr_scheduler.StepLR, scheduler_params={}):
        """
        [object]_params : dict
            Must include values wrapped in a list for all default parameters of [object] 
            ----
            Example:
                scheduler_params={'step_size': [7], 'gamma': [0.1, 0.01, 0.001]}
        """
        self.loss = loss
        self.optimizer = optimizer
        self.optimizer_params = optimizer_params
        self.scheduler = scheduler
        self.scheduler_params = scheduler_params
        self.best_model = ...
        self.best_acc = 0
        
    def tune(self):
        
        self.param_grid_ = list(product(*{**self.optimizer_params, **self.scheduler_params}.values()))
        self.param_grid_score_ = {}
        
        self.param_grid_len_ = len(self.param_grid_)
        for (i, param_set) in enumerate(self.param_grid_):
            print('Parameters set {}/{}'.format(i, self.param_grid_len_), end='\n')
            print(param_set)

            self.model_ = make_densenet()
            
            if self.optimizer == torch.optim.SGD:
                self.optimizer_ = self.optimizer(self.model_.parameters(), *param_set[:len(self.optimizer_params)])
            elif self.optimizer == torch.optim.Adam:
                self.optimizer_ = self.optimizer(self.model_.parameters(), *param_set[:len(self.optimizer_params)])
            else:
                raise RuntimeError('unknown type of optimizer')
                
            self.scheduler_ = self.scheduler(self.optimizer_, *param_set[len(self.optimizer_params):])

            _, best_model, score = train_model(self.model_, self.loss, self.optimizer_, self.scheduler_, n_epochs=26)
            if score > self.best_acc:
                self.best_model = copy.deepcopy(best_model)
                self.best_acc = score
            else:
                del best_model
#             self.current_score_ = sum(score[-3:]) / 3
            
#             self.param_grid_score_[str(param_set)[1:-1]] = self.current_score_
#         self.best_score_ = max(self.param_grid_score_.values())
#         self.best_params_ = max(self.param_grid_score_, key=lambda key: self.param_grid_score_[key])

In [0]:
import copy
def train_model(model, loss, optimizer, scheduler, n_epochs):
    best_acc = 0
    validation_accuracy = []
    
    for epoch in range(n_epochs):
        print('Epoch {}:'.format(epoch))
        
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_loader
#                 scheduler.step()
                model.train()
            else:
                dataloader = val_loader
                model.eval()
                
            batch_loss = 0.
            batch_acc = 0.
            
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_val = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)
                    
                    if phase == 'train':
                        loss_val.backward()
                        optimizer.step()
                        scheduler.step()
                        
                batch_loss += loss_val.item()
                batch_acc += (preds_class == labels.data).float().mean().item()
                
            epoch_loss = batch_loss / len(dataloader)
            epoch_acc = batch_acc / len(dataloader)
            
            if phase == 'val':
                validation_accuracy.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_model = copy.deepcopy(model)
                    best_acc = epoch_acc
        
            print('{} Loss: {:.3f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
    return model, best_model, best_acc    

In [0]:
# child_counter = 0
# for child in model.children():
#     print('child {} is'.format(child_counter))
#     print(child)
#     child_counter += 1

In [0]:
# child_counter = 0
# child_of_child_counter = 0

# for child in model.children():
#     for child_child in child.children():
#         print('child {} of child {} is '.format(child_of_child_counter, child_counter))
#         print(child_child)
#         child_of_child_counter += 1
#     child_of_child_counter = 0
#     child_counter += 1

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
loss = torch.nn.CrossEntropyLoss()
optimizer_params={'lr': [0.01, 0.1],
                  'momentum': [0.9, 0],
                  'dampening': [0.01],
                  'weight_decay': [0.1, 0.0001, 1],
                  'nesterov': [False]
                 }
scheduler_params = {'T_max': [5, 15]}

model_cv = GridSearch(loss, optimizer=torch.optim.SGD, optimizer_params=optimizer_params,
                     scheduler=torch.optim.lr_scheduler.CosineAnnealingLR, scheduler_params=scheduler_params)

In [0]:
model_cv.tune()

Parameters set 0/24
(0.01, 0.9, 0.01, 0.1, False, 5)
Epoch 0:
train Loss: 0.613 Acc: 0.6250
val Loss: 0.432 Acc: 0.7868
Epoch 1:
train Loss: 0.406 Acc: 0.8250
val Loss: 0.317 Acc: 0.9500
Epoch 2:
train Loss: 0.240 Acc: 0.9750
val Loss: 0.313 Acc: 0.9461
Epoch 3:
train Loss: 0.216 Acc: 1.0000
val Loss: 0.277 Acc: 0.9461
Epoch 4:
train Loss: 0.141 Acc: 1.0000
val Loss: 0.176 Acc: 0.9553
Epoch 5:
train Loss: 0.038 Acc: 1.0000
val Loss: 0.126 Acc: 0.9566
Epoch 6:
train Loss: 0.015 Acc: 1.0000
val Loss: 0.118 Acc: 0.9539
Epoch 7:
train Loss: 0.006 Acc: 1.0000
val Loss: 0.112 Acc: 0.9553
Epoch 8:
train Loss: 0.008 Acc: 1.0000
val Loss: 0.114 Acc: 0.9579
Epoch 9:
train Loss: 0.007 Acc: 1.0000
val Loss: 0.129 Acc: 0.9474
Epoch 10:
train Loss: 0.003 Acc: 1.0000
val Loss: 0.128 Acc: 0.9592
Epoch 11:
train Loss: 0.002 Acc: 1.0000
val Loss: 0.134 Acc: 0.9566
Epoch 12:
train Loss: 0.001 Acc: 1.0000
val Loss: 0.119 Acc: 0.9566
Epoch 13:
train Loss: 0.001 Acc: 1.0000
val Loss: 0.116 Acc: 0.9579
Epoch

In [0]:
import copy

In [0]:
model = copy.deepcopy(model_cv.best_model)

In [0]:
torch.save(model, 'densenet_un')

In [0]:
model_cv.best_acc

0.9789473712444305

In [0]:

optimizer = torch.optim.SGD(model.parameters(), lr=8e-3, momentum=0.9, dampening=0, weight_decay=0.05)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12, gamma=0.3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

In [0]:
model, best_model, val_acc = train_model(model, loss, optimizer, scheduler, n_epochs=100)

In [0]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'),
               os.path.join(test_dir, 'unknown'))

FileExistsError: ignored

In [0]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path, ))
        return tuple_with_path

In [0]:
test_dataset = ImageFolderWithPaths(test_dir, val_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=batch_size)

In [0]:
def evaluate(model, loader): 
    model.eval()

    test_predictions = []
    test_img_paths = []
    for inputs, _, paths in loader:
        inputs = inputs.to(device)
        with torch.set_grad_enabled(False):
            preds = model(inputs)
        test_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:, 1].data.cpu().numpy())
        test_img_paths.extend(paths)

    test_predictions = np.concatenate(test_predictions)
    return test_predictions, test_img_paths

In [0]:
test_predictions, test_img_paths = evaluate(model, test_loader)

In [0]:
len(test_predictions)

744

In [0]:
len(test_img_paths)

744

In [0]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths,
                                        'label': test_predictions})

In [0]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty'
                                                   if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)

In [0]:
submission_df.head(6)

,label
id,
0000,dirty
0001,dirty
0002,dirty
0003,dirty
0004,dirty
0005,dirty


In [0]:
submission_df.to_csv('submission.csv')